In [ ]:
# Atree is the matrix 
# Atree[::2,::2,::2]

In [ ]:
#100 GOLD STANDARD CODE ALL IN ONE

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT



COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")




#2 Function to write data to Word document
def write_to_word(text, bold=False, indent_level=0, font_color=None):
    paragraph = doc.add_paragraph(text)
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    run = paragraph.runs[0]
    if bold:
        run.bold = True
    font = run.font
    font.name = 'Courier New'
    font.size = Pt(12)
    if font_color:
        # Check if the font_color is a named color
        if isinstance(font_color, str) and font_color in COLORS_DICT:
            r, g, b = COLORS_DICT[font_color]
        else:
            r, g, b = font_color
        font.color.rgb = RGBColor(r, g, b)
  

#3

def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")



#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=5, font_color='orange')
    write_to_word(spacing_output, indent_level=5, font_color='orange')




    #5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None
    

#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None


#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

#8 Visualization

def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        

        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))


        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

   
        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 


#9 Print Filtered Nrrd Header

def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=8, font_color='orange')

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"Segment {seg_num} LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=10, font_color=(r,g,b))  # Write the segment name without '■'
            
            # Add other attributes
            write_to_word(segment_id, indent_level=10)
            write_to_word(segment_color, indent_level=10)
            write_to_word(segment_label_value, indent_level=10)
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Segment {seg_num} Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=10)

            write_to_word(" ")
    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)




#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path):

    files_and_subdirs = os.listdir(folder_path)
    output_text = f"Folder Path: {folder_path}"
    print(output_text)
    write_to_word(output_text, bold=True)
    write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files/Folders NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=10, font_color='blue')

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"Processing: {item}"
                output_filename_NIFTI = item.replace("S-", "SS-").replace(".nrrd", ".nii.gz").replace(".seg.",".")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=5)
                write_to_word(output_filename_save, bold=True, indent_level=6, font_color='red')
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=5, font_color='orange')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                write_to_word(' ')
                


                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=3)     



                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=8, font_color=font_color_val)
                        for i in range(num_segments):
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"  Name: {segment_names[i]}",
                                f"  Label Value: {label_values[i]}"
                                f"  Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=10, font_color=font_color_val)
                        write_to_word(" ")

                            
                else:
                    print_filtered_nrrd_header(image, data)



#11 Search and Process Files 
def search_and_process(base_path, max_cases=1):
    processed_cases = 0  # Counter to keep track of processed cases
    
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            processed_cases += 1  # Increment the counter
                            if processed_cases >= max_cases:
                                return 

        
    #12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx')

In [1]:
import os

# Define the directory path
folder_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate"

# Use the os.listdir() function to get a list of all files and subdirectories
files_and_subdirs = os.listdir(folder_path)

# Iterate through the list and print the file names
for item in files_and_subdirs:
    if os.path.isfile(os.path.join(folder_path, item)):  # Check if it's a file and not a subdirectory
        print(item)

S-Final-1-label.nrrd
S-Final-1.seg.nrrd
S-Seeds-1.seg.nrrd
S-Seeds-2.seg.nrrd
S-Segmentation-1.seg.nrrd
S-Segmentation-2.seg.nrrd
S-Segmentation-3.seg.nrrd
S-Segmentation-4.seg.nrrd
S-Segmentation-5.seg.nrrd
Vx3.nrrd


In [11]:
from mayavi import mlab
def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

In [ ]:
# COUNTOUR 3D takes a long time

import SimpleITK as sitk
from mayavi import mlab
import numpy as np

def update_camera_info(vtk_obj, event):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

def read_nrrd_image(file_path):
    reader = sitk.ImageFileReader()
    reader.SetFileName(file_path)
    image = reader.Execute()
    data = sitk.GetArrayFromImage(image)
    return data

def create_mayavi_viewer(data, figure_name):
    mlab.figure(figure_name, bgcolor=(0, 0, 0))

    # Extract the two volumes based on their scalar values
    data1 = np.where(data == 1, 1, 0)
    data2 = np.where(data == 2, 1, 0)

    # Create contours for the volumes with different colors
    contour1 = mlab.contour3d(data1, colormap="spectral", opacity=1)
    contour2 = mlab.contour3d(data2, colormap="hot", opacity=1)

    # Set the camera settings
    mlab.gcf().scene.camera.position = [1766, 600, 200]
    mlab.gcf().scene.camera.focal_point = [297, 238, 302]
    mlab.gcf().scene.camera.view_up = [0, 1, 0]

    # Attach the update_camera_info function to the mouse move event
    mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

# Define your .nrrd files for the models
model_paths = ["z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate/S-Final-1-label.nrrd", 
               "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate/S-Segmentation-1-label.nrrd"]

for i, model_path in enumerate(model_paths):
    data = read_nrrd_image(model_path)
    figure_name = f"Figure_{i+1}"
    create_mayavi_viewer(data, figure_name)

mlab.show()


In [ ]:
#SCALAR FIELD WAY FASTER BUT COLOR IS WRONG

import SimpleITK as sitk
from mayavi import mlab
import numpy as np

def update_camera_info(vtk_obj, event):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

def read_nrrd_image(file_path):
    reader = sitk.ImageFileReader()
    reader.SetFileName(file_path)
    image = reader.Execute()
    data = sitk.GetArrayFromImage(image)
    return data

def create_mayavi_viewer(data1, data2, figure_name):
    mlab.figure(figure_name, bgcolor=(0, 0, 0))

    # Create volume rendering for data with value 1 (blue)
    volume1 = mlab.pipeline.volume(mlab.pipeline.scalar_field(data1), color=(0, 0, 1))  # Blue color

    # Create volume rendering for data with value 2 (red)
    volume2 = mlab.pipeline.volume(mlab.pipeline.scalar_field(data2), color=(1, 0, 0))  # Red color

    # Set the camera settings
    mlab.gcf().scene.camera.position = [1766, 600, 200]
    mlab.gcf().scene.camera.focal_point = [297, 238, 302]
    mlab.gcf().scene.camera.view_up = [0, 1, 0]

    # Attach the update_camera_info function to the mouse move event
    mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

# Define your .nrrd files for the two volumes
data1_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate/S-Final-1-label.nrrd"
data2_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate/S-Segmentation-1-label.nrrd"

# Read the data from the files
data1 = read_nrrd_image(data1_path)
data2 = read_nrrd_image(data2_path)

# Specify a unique figure name for each volume
figure_name1 = "Figure_1"
figure_name2 = "Figure_2"

# Create Mayavi viewers for both volumes
create_mayavi_viewer(data1, data2, figure_name1)
create_mayavi_viewer(data2, data1, figure_name2)  # Swap data1 and data2 for the second viewer to differentiate

# Show both viewers
mlab.show()


In [17]:
#BEST VISUALIZATION TECHNIQUE CURRENTLY
import SimpleITK as sitk
from mayavi import mlab
import numpy as np
from mayavi.modules.surface import Surface
from tvtk.api import tvtk

def update_camera_info(vtk_obj, event):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

def read_nrrd_image(file_path):
    reader = sitk.ImageFileReader()
    reader.SetFileName(file_path)
    image = reader.Execute()
    data = sitk.GetArrayFromImage(image)
    return data

# def create_mayavi_viewer(data, figure_name):
    mlab.figure(figure_name, bgcolor=(0, 0, 0))

    # Create volume rendering for data with value 1 (red) and value 2 (blue)
    volume = mlab.pipeline.volume(mlab.pipeline.scalar_field(data), color=(1, 0, 0))

    # Set the camera settings
    mlab.gcf().scene.camera.position = [1766, 600, 200]
    mlab.gcf().scene.camera.focal_point = [297, 238, 302]
    mlab.gcf().scene.camera.view_up = [0, 1, 0]

    # Attach the update_camera_info function to the mouse move event
    mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
# def create_mayavi_viewer(data, figure_name):
    mlab.figure(figure_name, bgcolor=(0, 0, 0))
    
    # Create the scalar field
    src = mlab.pipeline.scalar_field(data)
    
    # Create contour for data with value 1 (Red)
    contour1 = mlab.pipeline.contour(src)
    contour1.filter.contours = [1]
    surface1 = mlab.pipeline.surface(contour1)
    surface1.actor.property.color = (1, 0, 0)  # Red
    surface1.actor.property.opacity = 0.5
    
    # Create contour for data with value 2 (Blue)
    contour2 = mlab.pipeline.contour(src)
    contour2.filter.contours = [2]
    surface2 = mlab.pipeline.surface(contour2)
    surface2.actor.property.color = (0, 0, 1)  # Blue
    surface2.actor.property.opacity = 0.5

    # Camera settings
    mlab.gcf().scene.camera.position = [1766, 600, 200]
    mlab.gcf().scene.camera.focal_point = [297, 238, 302]
    mlab.gcf().scene.camera.view_up = [0, 1, 0]

    # Attach the update_camera_info function to the mouse move event
    mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

def create_mayavi_viewer(data, figure_name):
    mlab.figure(figure_name, bgcolor=(0, 0, 0))
    
    # Mask data for value 1 and 2
    data1 = np.where(data == 1, data, 0)
    data2 = np.where(data == 2, data, 0)
    
    src1 = mlab.pipeline.scalar_field(data1)
    src2 = mlab.pipeline.scalar_field(data2)

    # Create an iso surface for data1 (value 1) and set its color to Red
    iso_surface1 = mlab.pipeline.iso_surface(src1, contours=[1], color=(0, .59, .8078), opacity=1)
    iso_surface2 = mlab.pipeline.iso_surface(src2, contours=[2], color=(.847, .396, .3098), opacity=1)
    
    # Set the camera settings
    mlab.gcf().scene.camera.position = [1766, 600, 200]
    mlab.gcf().scene.camera.focal_point = [297, 238, 302]
    mlab.gcf().scene.camera.view_up = [0, 1, 0]

    # Attach the update_camera_info function to the mouse move event
    mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

# Define your .nrrd file for the volume
data_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate/S-Final-1-label.nrrd"

# Read the data from the file
data = read_nrrd_image(data_path)

# Specify a unique figure name
figure_name = "Single Figure"

# Create a Mayavi viewer for the volume
create_mayavi_viewer(data, figure_name)

# Show the viewer
mlab.show()


In [13]:
import os
import SimpleITK as sitk

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Define the directory path
folder_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate"

# Use the os.listdir() function to get a list of all files and subdirectories
files_and_subdirs = os.listdir(folder_path)

# Iterate through the list, check if it's a file, and process the .nrrd files
for item in files_and_subdirs:
    if os.path.isfile(os.path.join(folder_path, item)) and item.endswith('.nrrd'):  
        file_path = os.path.join(folder_path, item)
        print(f"Processing: {item}")
        num_segments, segment_names, label_values = extract_segment_info(file_path)

        if num_segments is not None:
            print(f"Number of Segments: {num_segments}")
            for i in range(num_segments):
                print(f"Segment {i + 1}:")
                print(f"  Name: {segment_names[i]}")
                print(f"  Label Value: {label_values[i]}\n")


Processing: S-Final-1-label.nrrd
Number of Segments: 0
Processing: S-Final-1.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: Aseed
  Label Value: 1

Segment 2:
  Name: Vseed
  Label Value: 2

Segment 3:
  Name: Miscellaneous
  Label Value: 3

Processing: S-Seeds-1.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 2

Segment 2:
  Name: V
  Label Value: 1

Segment 3:
  Name: fat
  Label Value: 3

Processing: S-Seeds-2.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 2

Segment 2:
  Name: V
  Label Value: 1

Segment 3:
  Name: fat
  Label Value: 3

Processing: S-Segmentation-1-label.nrrd
Number of Segments: 0
Processing: S-Segmentation-1.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 1

Segment 2:
  Name: V
  Label Value: 2

Segment 3:
  Name: fat
  Label Value: 3

Processing: S-Segmentation-2.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 1

Segment 2:
  Name: V
  Label Value: 2

Segment 3:
  Name: fat
  Label V

In [15]:
# Works for one Case
import os
import SimpleITK as sitk
import numpy as np

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

def extract_segments_info_for_labels(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get the pixel array from the image
        data = sitk.GetArrayFromImage(image)

        # Extract unique segment values (e.g., 0s, 1s, and 2s)
        unique_values = np.unique(data)

        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Define the directory path
folder_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-81-Spiromics-512592363/Markups/88-Nate"

# Use the os.listdir() function to get a list of all files and subdirectories
files_and_subdirs = os.listdir(folder_path)

# Iterate through the list, check if it's a file, and process the .nrrd files
for item in files_and_subdirs:
    if os.path.isfile(os.path.join(folder_path, item)) and item.endswith('.nrrd'):
        file_path = os.path.join(folder_path, item)
        print(f"Processing: {item}")

        if '-label.nrrd' in item:
            num_segments, segment_names, label_values = extract_segments_info_for_labels(file_path)
        else:
            num_segments, segment_names, label_values = extract_segment_info(file_path)

        if num_segments is not None and num_segments > 0:
            print(f"Number of Segments: {num_segments}")
            for i in range(num_segments):
                print(f"Segment {i + 1}:")
                print(f"  Name: {segment_names[i]}")
                print(f"  Label Value: {label_values[i]}\n")


Processing: S-Final-1-label.nrrd
Number of Segments: 3
Segment 1:
  Name: Segment_0
  Label Value: 0

Segment 2:
  Name: Segment_1
  Label Value: 1

Segment 3:
  Name: Segment_2
  Label Value: 2

Processing: S-Final-1.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: Aseed
  Label Value: 1

Segment 2:
  Name: Vseed
  Label Value: 2

Segment 3:
  Name: Miscellaneous
  Label Value: 3

Processing: S-Seeds-1.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 2

Segment 2:
  Name: V
  Label Value: 1

Segment 3:
  Name: fat
  Label Value: 3

Processing: S-Seeds-2.seg.nrrd
Number of Segments: 3
Segment 1:
  Name: A
  Label Value: 2

Segment 2:
  Name: V
  Label Value: 1

Segment 3:
  Name: fat
  Label Value: 3

Processing: S-Segmentation-1-label.nrrd
Number of Segments: 3
Segment 1:
  Name: Segment_0
  Label Value: 0

Segment 2:
  Name: Segment_1
  Label Value: 1

Segment 3:
  Name: Segment_2
  Label Value: 2

Processing: S-Segmentation-1.seg.nrrd
Number of Segments: 3
Segment

In [16]:
# ADD S- to make search more efficient
# Print Color of RGB or box of the color that specific ■
# Do not go duplicates folder
# search through both 
# create excel or word documnet to show how many files and do prifiling within each case
# once you read and unravel
# Save all considered files under one of these three
# If there are multiple finals then rename them as SS-Segementations-n+1.nii.gz
# SS-Case-34-Seeds-2-Label.nii.gz  xN
# SS-Case-34-Segmentation-7-Label.nii.gz  xM
# SS-Case-34-Final-Label.nii.gz 
# if s-final date of mofied is > s-finalseg by 3 days: WARN*
# Create snapshot as final golden standard of quality assurance
# Want to potentially run this code often because we may be correct some of the segmentations
# Also may want to save these to a different location than D-Images

import os
import SimpleITK as sitk
import numpy as np

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

def extract_segments_info_for_labels(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get the pixel array from the image
        data = sitk.GetArrayFromImage(image)

        # Extract unique segment values (e.g., 0s, 1s, and 2s)
        unique_values = np.unique(data)

        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None
def process_nrrd_files_in_folder(folder_path):
    # Use the os.listdir() function to get a list of all files and subdirectories
    files_and_subdirs = os.listdir(folder_path)

    # Iterate through the list, check if it's a file, and process the .nrrd files
    for item in files_and_subdirs:
        if os.path.isfile(os.path.join(folder_path, item)) and item.endswith('.nrrd'):
            file_path = os.path.join(folder_path, item)
            print(f"Processing: {item}")

            if '-label.nrrd' in item:
                num_segments, segment_names, label_values = extract_segments_info_for_labels(file_path)
            else:
                num_segments, segment_names, label_values = extract_segment_info(file_path)

            if num_segments is not None and num_segments > 0:
                print(f"Folder Path: {folder_path}")
                print(f"Number of Segments: {num_segments}")
                for i in range(num_segments):
                    print(f"Segment {i + 1}:")
                    print(f"  Name: {segment_names[i]}")
                    print(f"  Label Value: {label_values[i]}\n")

def search_and_process(base_path):
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))

# Define the directory path
base_folder_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done"
search_and_process(base_folder_path)

Processing: S-Final-1.nrrd
Folder Path: z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done\Case-22-Spiromics-53062428\MarkUps\88-Jensen
Number of Segments: 3
Segment 1:
  Name: A Seed
  Label Value: 1

Segment 2:
  Name: V Seed
  Label Value: 2

Segment 3:
  Name: Misc
  Label Value: 3

Processing: S-Final-2-label.nrrd
Folder Path: z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done\Case-22-Spiromics-53062428\MarkUps\88-Jensen
Number of Segments: 3
Segment 1:
  Name: Segment_0
  Label Value: 0

Segment 2:
  Name: Segment_1
  Label Value: 1

Segment 3:
  Name: Segment_2
  Label Value: 2

Processing: S-Final-2.nrrd
Folder Path: z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done\Case-22-Spiromics-53062428\MarkUps\88-Jensen
Number of Segments: 3
Segment 1:
  Name: A Seed
  Label Value: 1

Segment 2:
  Name: V Seed
  Label Value: 2

Segment 3:
  Name: Misc
  Label Value: 3

Processing: S-Seed-1.nrrd
Folder Path: z:/D-Images/SPIROMICS-SubStudy/2-Results-

In [13]:
import SimpleITK as sitk

def print_nrrd_header(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        keys = image.GetMetaDataKeys()

        # Print the header information
        print("NRRD Header Information:")
        for key in keys:
            value = image.GetMetaData(key)
            print(f"{key}: {value}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
file_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-03-MESAL-6011969-E3/Markups/88-Jensen/S-Final-2.seg.nrrd"  # Replace with the path to your .nrrd file
print_nrrd_header(file_path)


NRRD Header Information:
ITK_InputFilterName: NrrdImageIO
ITK_original_direction: [UNKNOWN_PRINT_CHARACTERISTICS]

ITK_original_spacing: [UNKNOWN_PRINT_CHARACTERISTICS]

NRRD_kinds[0]: domain
NRRD_kinds[1]: domain
NRRD_kinds[2]: domain
NRRD_space: left-posterior-superior
Segment0_Color: 0.901961 0.862745 0.27451
Segment0_ColorAutoGenerated: 1
Segment0_Extent: 0 676 0 501 0 598
Segment0_ID: Segment_1
Segment0_LabelValue: 2
Segment0_Layer: 0
Segment0_Name: Misc
Segment0_NameAutoGenerated: 0
Segment0_Tags: Segmentation.Status:inprogress|TerminologyEntry:Segmentation category and type - 3D Slicer General Anatomy list~SCT^85756007^Tissue~SCT^248300009^Body fat~^^~Anatomic codes - DICOM master list~^^~^^|
Segment1_Color: 0 0.592157 0.807843
Segment1_ColorAutoGenerated: 1
Segment1_Extent: 0 676 0 501 0 598
Segment1_ID: Segment_1_5_2_2
Segment1_LabelValue: 3
Segment1_Layer: 0
Segment1_Name: A1
Segment1_NameAutoGenerated: 0
Segment1_Tags: Segmentation.Status:inprogress|TerminologyEntry:Segmenta

In [10]:
import SimpleITK as sitk

def print_nrrd_header(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        keys = image.GetMetaDataKeys()

        # Print the header information
        print("NRRD Header Information:")
        for key in keys:
            value = image.GetMetaData(key)
            print(f"{key}: {value}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
file_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-03-MESAL-6011969-E3/Markups/88-Jensen/S-Final-2-label.nrrd"
print_nrrd_header(file_path)


NRRD Header Information:
ITK_InputFilterName: NrrdImageIO
ITK_original_direction: [UNKNOWN_PRINT_CHARACTERISTICS]

ITK_original_spacing: [UNKNOWN_PRINT_CHARACTERISTICS]

NRRD_kinds[0]: domain
NRRD_kinds[1]: domain
NRRD_kinds[2]: domain
NRRD_space: left-posterior-superior


In [8]:
import SimpleITK as sitk

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Example usage
file_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-03-MESAL-6011969-E3/Markups/88-Jensen/99-Old Segmentation/S-Segmentation-2.nrrd"  # Replace with the path to your .nrrd file
num_segments, segment_names, label_values = extract_segment_info(file_path)

if num_segments is not None:
    print(f"Number of Segments: {num_segments}")
    for i in range(num_segments):
        print(f"Segment {i + 1}:")
        print(f"  Name: {segment_names[i]}")
        print(f"  Label Value: {label_values[i]}\n")


Number of Segments: 5
Segment 1:
  Name: ASeed
  Label Value: 1

Segment 2:
  Name: MSeed
  Label Value: 3

Segment 3:
  Name: VSeed
  Label Value: 1

Segment 4:
  Name: Segment_5
  Label Value: 1

Segment 5:
  Name: Segment_6
  Label Value: 2



In [20]:
import SimpleITK as sitk

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Example usage
file_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-03-MESAL-6011969-E3/Markups/88-Jensen/S-Final-2.seg.nrrd"  # Replace with the path to your .nrrd file
num_segments, segment_names, label_values = extract_segment_info(file_path)

if num_segments is not None:
    print(f"Number of Segments: {num_segments}")
    for i in range(num_segments):
        print(f"Segment {i + 1}:")
        print(f"  Name: {segment_names[i]}")
        print(f"  Label Value: {label_values[i]}\n")


Number of Segments: 3
Segment 1:
  Name: Misc
  Label Value: 2

Segment 2:
  Name: A1
  Label Value: 3

Segment 3:
  Name: V1
  Label Value: 4



In [14]:
# Works for labels 
import SimpleITK as sitk
import numpy as np

def extract_segments_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get the pixel array from the image
        data = sitk.GetArrayFromImage(image)

        # Extract unique segment values (0s, 1s, and 2s)
        unique_values = np.unique(data)

        # Print segment information
        print("Segments Information:")
        for value in unique_values:
            segment_name = f"Segment_{value}"
            print(f"Segment Name: {segment_name}")
            print(f"Segment Label Value: {value}\n")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
file_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-03-MESAL-6011969-E3/Markups/88-Jensen/S-Final-2-label.nrrd"  # Replace with the path to your .nrrd file
extract_segments_info(file_path)


Segments Information:
Segment Name: Segment_0
Segment Label Value: 0

Segment Name: Segment_1
Segment Label Value: 1

Segment Name: Segment_2
Segment Label Value: 2



In [17]:
#CASE 84 segmentation

import SimpleITK as sitk

def extract_segment_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get all metadata keys
        metadata_keys = image.GetMetaDataKeys()

        # Filter keys related to segments
        segment_keys = [key for key in metadata_keys if key.startswith("Segment") and key.endswith("_LabelValue")]

        # Extract segment information
        num_segments = len(segment_keys)
        segment_names = []
        label_values = []

        for key in segment_keys:
            # Extract segment name
            segment_name_key = key.replace("_LabelValue", "_Name")
            segment_name = image.GetMetaData(segment_name_key)
            segment_names.append(segment_name)

            # Extract segment label value
            label_value = int(image.GetMetaData(key))
            label_values.append(label_value)

        return num_segments, segment_names, label_values

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Example usage
file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-84-Spiromics-53053170/Markups/88-Nate/S-Final-1.seg.nrrd"  # Replace with the path to your .nrrd file
num_segments, segment_names, label_values = extract_segment_info(file_path)

if num_segments is not None:
    print(f"Number of Segments: {num_segments}")
    for i in range(num_segments):
        print(f"Segment {i + 1}:")
        print(f"  Name: {segment_names[i]}")
        print(f"  Label Value: {label_values[i]}\n")


Number of Segments: 3
Segment 1:
  Name: Aseed
  Label Value: 1

Segment 2:
  Name: Vseed
  Label Value: 2

Segment 3:
  Name: Miscellaneous
  Label Value: 3



In [16]:
#CASE 84 LABEL NEW CREATED
import SimpleITK as sitk
import numpy as np

def extract_segments_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get the pixel array from the image
        data = sitk.GetArrayFromImage(image)

        # Extract unique segment values (0s, 1s, and 2s)
        unique_values = np.unique(data)

        # Print segment information
        print("Segments Information:")
        for value in unique_values:
            segment_name = f"Segment_{value}"
            print(f"Segment Name: {segment_name}")
            print(f"Segment Label Value: {value}\n")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-84-Spiromics-53053170/Markups/88-Nate/S-Final-1-label_3.nrrd"  # Replace with the path to your .nrrd file
extract_segments_info(file_path)


Segments Information:
Segment Name: Segment_0
Segment Label Value: 0

Segment Name: Segment_1
Segment Label Value: 1

Segment Name: Segment_2
Segment Label Value: 2



In [19]:
# CASE 84 LABEL TEST FAKE

import SimpleITK as sitk
import numpy as np

def extract_segments_info(file_path):
    try:
        # Read the .nrrd file
        image = sitk.ReadImage(file_path)

        # Get the pixel array from the image
        data = sitk.GetArrayFromImage(image)

        # Extract unique segment values (0s, 1s, and 2s)
        unique_values = np.unique(data)

        # Print segment information
        print("Segments Information:")
        for value in unique_values:
            segment_name = f"Segment_{value}"
            print(f"Segment Name: {segment_name}")
            print(f"Segment Label Value: {value}\n")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage
file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-84-Spiromics-53053170/Markups/88-Nate/S-Final-1_1-label-TESTFAKE.nrrd"  # Replace with the path to your .nrrd file
extract_segments_info(file_path)


Segments Information:
Segment Name: Segment_0
Segment Label Value: 0

Segment Name: Segment_1
Segment Label Value: 1

Segment Name: Segment_2
Segment Label Value: 2

Segment Name: Segment_3
Segment Label Value: 3

